In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
df=pd.read_csv('final_estimation_file_apta_added.csv',
                    encoding='utf-8')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [3]:
tsd_data=pd.read_csv('tsd_pop_employment.csv',
                    encoding='utf-8')
tsd_data.drop(columns=['Unnamed: 0','CBSA Title'],inplace=True)
tsd_data.head()

,CBSA Code,STATE,Year,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10180,Texas,2002,9489.0,57166.0,58397.0,0.0,0.0,0.0,23086.0,25800.0,9511.0,48886.0,58397.0,0.0,0.0,1216.0,2355.0,0.0,0.0,0.0,954.0,1065.0,336.0,2019.0,2355.0,0.0,0.0
1,10180,Texas,2003,9484.0,56917.0,57221.0,0.0,0.0,0.0,22049.0,25074.0,10098.0,47123.0,57221.0,0.0,0.0,1216.0,2462.0,0.0,0.0,0.0,1032.0,1069.0,361.0,2101.0,2462.0,0.0,0.0
2,10180,Texas,2004,9599.0,59025.0,61419.0,0.0,0.0,0.0,23062.0,27199.0,11158.0,50261.0,61419.0,0.0,0.0,1216.0,2970.0,0.0,0.0,0.0,1231.0,1315.0,424.0,2546.0,2970.0,0.0,0.0
3,10180,Texas,2005,9645.0,57879.0,60001.0,0.0,0.0,0.0,21854.0,26081.0,12066.0,47935.0,60001.0,0.0,0.0,1216.0,2938.0,0.0,0.0,0.0,1134.0,1326.0,478.0,2460.0,2938.0,0.0,0.0
4,10180,Texas,2006,9819.0,59588.0,62311.0,0.0,0.0,0.0,22008.0,26639.0,13664.0,48647.0,62311.0,0.0,0.0,1216.0,2983.0,0.0,0.0,0.0,1122.0,1342.0,519.0,2464.0,2983.0,0.0,0.0


In [4]:
df=pd.merge(df,tsd_data,left_on=['CBSA','Year'],right_on=['CBSA Code','Year'],how='left')
df=df.drop_duplicates()
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,2002,2018,0.114148,NaN,NaN,NaN,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,309488.2031,87.4,6.2,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.159808,2417.239652,both,24,10420.0,Ohio,81463.0,314749.0,305401.0,0.0,0.0,0.0,92556.0,127165.0,85680.0,219721.0,305401.0,0.0,0.0,13619.2,54517.0,0.0,0.0,0.0,18228.0,25397.0,10892.0,43625.0,54517.0,0.0,0.0
1,10420,"Akron, OH Metro Area",Bus,2003,5395901.888,3976568.624,2.318587e+06,2002,2018,0.429694,NaN,NaN,NaN,659397.0625,504631.8125,137582.3125,753.12500,25120.18750,25632.68750,25995.68750,10350.875,682850.56,8.81,8.66,82.79,24.33,63.0,12.68,269144.19,22073.44,67400.94,115443.00,44874.88,19351.94,8.20,43566.68750,56379.7500,39.24,46.66,14.10,23342.50,348574.17,371916.67,6.28,93.72,1.55,NaN,119199.0,14.71,1.37,2014,http://www.crainscleveland.com/article/2014082...,0,0,15690640.83,34414.65688,35116.78188,35614.09188,59686.36188,77240.2575,2.1235,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,316497.1875,87.2,6.4,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.588681,2417.305061,both,24,10420.0,Ohio,79781.0,313075.0,318486.0,0.0,0.0,0.0,95379.0,128948.0,94159.0,224327.0,318486.0,0.0,0.0,13619.2,51976.0,0.0,0.0,0.0,17093.0,23767.0,11116.0,40860.0,51976.0,0.0,0.0
2,10420,"Akron, OH Metro Area",Bus,2004,5907162.281,4315392.555,2.279211e+06,2002,2018,0.385839,NaN,NaN,NaN,672492.7500,513183.7500,138058.7500,1942.50000,24804.25000,252

In [5]:
df['TSD_EMP_PCT_LEHD']=100*df['TOT_EMP_TSD_TRACTS']/df['TOT_EMP_ALL_TRACTS']
df['TSD_EMP_PCT_CBSA']=100*df['TOT_EMP_TSD_TRACTS']/df['TOT_EMP_MSA']
df['TSD_POP_PCT_LEHD']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS'] #RAC data
df['TSD_POP_PCT_CBSA']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['Tot_Pop']
df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_RA']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_1250_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_RA']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_BETN_1250_3333_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_RA']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_3333_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']

In [6]:
# read in and join the maintenance and restructure data
mr=pd.read_csv('maintenance_restructure.csv',
                    encoding='utf-8')
mr = mr[['CBSA', 'Mode', 'Year', 'MAINTENANCE', 'MAINTENANCE_NYC', 'MAINTENANCE_WMATA', 'RESTRUCTURE']]

df = df.merge(mr, on=['CBSA', 'Mode', 'Year'], how='left')

In [7]:
# read in mechanical and other failures

failures=pd.read_csv('maintenance_final.csv',encoding='utf-8')

failures.drop(columns=['Unnamed: 0'],inplace=True)

df=df.merge(failures, on=['CBSA','MNAME','Mode','Year'],how='left')

In [8]:
df=df.rename(columns={'UPT_ADJUSTED':'UPT_ADJ','VRM_ADJUSTED':'VRM_ADJ','FARE_ADJUSTED':'FARE_ADJ','AVG_FARE_2018':'FARE_per_UPT_2018',
                     'Mecha_Failures':'Mechanical_Failures'})

In [9]:
df['Mechanical_Failures']=df['Mechanical_Failures'].fillna(0)
df['Total_Failures']=df['Total_Failures'].fillna(0)
df['MDBF_Mechanical']=np.where(df['Mechanical_Failures']>0,df['VRM_ADJ']/df['Mechanical_Failures'],df['VRM_ADJ'])
df['MDBF_Total']=np.where(df['Total_Failures']>0,df['VRM_ADJ']/df['Total_Failures'],df['VRM_ADJ'])

In [10]:
# read in and join the bike share updates
bsu=pd.read_csv('bike_share_update.csv',
                    encoding='utf-8')
bsu = bsu[['CBSA', 'Mode', 'Year', 'PBS_Flag_Update']]

df = df.merge(bsu, on=['CBSA', 'Mode', 'Year'], how='left')

#df['PBS_Flag_Missing'] = df['PBS_Flag'].apply(lambda x : x.isnan())
df['PBS_Flag'] = np.where(df['PBS_Flag'].isnull(), df['PBS_Flag_Update'], df['PBS_Flag'])

In [11]:
# 4 apta clusers
df['CLUSTER_APTA4'] = np.floor(df['CLUSTER_APTA']/10)

In [12]:
# exclude if ridership is zero--missing data
df = df[df['UPT_ADJ']>0]

In [13]:
# fill in missing data as needed

# min fare is zero
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : max(x,0))

# max fare is $20
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : min(x,20))

# zero vehicle HH can't go negative
df['HH_0Veh'] = df['HH_0Veh'].apply(lambda x : max(x,0))

In [14]:
df=df.drop_duplicates(['CBSA','Mode','Year'])

In [15]:
# exclude problematic data
old_len = len(df)
df = df[df['VRM_ADJ']>0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with missing VRM') 

# exclude if problem flag 2 is set
old_len = len(df)
df = df[df['PROBLEM_FLAG2']!=1]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with problem flag set to 1') 

# exclude rail in low op-ex group
old_len = len(df)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)
df['LOW_OPEX_RAIL'] = df['LOW_OPEX'] * df['RAIL_FLAG']
df = df[df['LOW_OPEX_RAIL']==0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with that have rail in the low op-ex group') 



Excluding 2 records with missing VRM
Excluding 36 records with problem flag set to 1
Excluding 48 records with that have rail in the low op-ex group


In [16]:
# exclude discontinuous data
df = df.sort_values(by=['CBSA', 'Mode', 'Year'])

excluded_records = 100
while excluded_records > 0: 
    old_len = len(df)
    df['YearDiff'] = df['Year'].shift(-1) - df['Year']
    df = df[(df['YearDiff']==1) | (df['Year']==2018)]
    excluded_records = old_len - len(df)
    print('Excluding ' + str(excluded_records) + ' records with discontinuities in year.') 




Excluding 1 records with discontinuities in year.
Excluding 1 records with discontinuities in year.
Excluding 0 records with discontinuities in year.


In [17]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [18]:
# include population + employment
df['POP_EMP'] = df['Tot_Pop'] + df['TOT_EMP_MSA']
df['HH_EMP'] = df['Total_HH'] + df['TOT_EMP_MSA']


# percent of pop born outside USA
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [19]:
# time effects
df['YEARS_SINCE_2002']  = df['Year'] - 2002

df['YEARS_2002_2010']  = df['Year'].apply(lambda x : min(x-2002, 8))
df['YEARS_AFTER_2010'] = df['Year'].apply(lambda x : max(x-2010, 0))

df['YEARS_2002_2014']  = df['Year'].apply(lambda x : min(x-2002, 12))
df['YEARS_AFTER_2014'] = df['Year'].apply(lambda x : max(x-2014, 0))


df['YEARS_SINCE_2002_BUS']  = df['YEARS_SINCE_2002'] * df['BUS_FLAG']
df['YEARS_SINCE_2002_RAIL'] = df['YEARS_SINCE_2002'] * df['RAIL_FLAG']

df['YEARS_2002_2010_BUS']   = df['YEARS_2002_2010'] * df['BUS_FLAG']
df['YEARS_AFTER_2010_BUS']  = df['YEARS_AFTER_2010']* df['BUS_FLAG']
df['YEARS_2002_2010_RAIL']  = df['YEARS_2002_2010'] * df['RAIL_FLAG']
df['YEARS_AFTER_2010_RAIL'] = df['YEARS_AFTER_2010']* df['RAIL_FLAG']

df['YEARS_2002_2014_BUS']   = df['YEARS_2002_2014'] * df['BUS_FLAG']
df['YEARS_AFTER_2014_BUS']  = df['YEARS_AFTER_2014']* df['BUS_FLAG']
df['YEARS_2002_2014_RAIL']  = df['YEARS_2002_2014'] * df['RAIL_FLAG']
df['YEARS_AFTER_2014_RAIL'] = df['YEARS_AFTER_2014']* df['RAIL_FLAG']

In [20]:
# more time effects

df['YEARS_AFTER_2012'] = df['Year'].apply(lambda x : max(x-2012, 0))

df['TNC_YEARS_AFTER_2012'] = df['YEARS_AFTER_2012'] * df['TNC_FLAG']

df['TNC_YEARS_AFTER_2012_BUS'] = df['TNC_YEARS_AFTER_2012'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2012_RAIL'] = df['TNC_YEARS_AFTER_2012'] * df['RAIL_FLAG']

In [21]:

df['YEARS_SINCE_TNC_2012'] = np.minimum(df['YEARS_SINCE_TNC'], df['YEARS_AFTER_2012'])

df['YEARS_SINCE_TNC_2012_BUS'] = df['YEARS_SINCE_TNC_2012'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_2012_RAIL'] = df['YEARS_SINCE_TNC_2012'] * df['RAIL_FLAG']

In [22]:
# test rail ramp up period
df['YEARS_SINCE_RAIL_START'] = (df['Year'] - df['min_year']) * df['RAIL_FLAG']
df['YEARS_SINCE_RAIL_START'] = np.where(df['min_year']==2002, 0, df['YEARS_SINCE_RAIL_START'])

df['YEARS_SINCE_RAIL_START_1'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 1))
df['YEARS_SINCE_RAIL_START_2'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_RAIL_START_3'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 3))



In [23]:
# calculate bus vs rail stuff

# various bus/rail specifics

df['TNC_FLAG_BUS'] = df['TNC_FLAG'] * df['BUS_FLAG']
df['TNC_FLAG_RAIL'] = df['TNC_FLAG'] * df['RAIL_FLAG']

df['YEARS_SINCE_TNC_BUS'] = df['YEARS_SINCE_TNC'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_RAIL'] = df['YEARS_SINCE_TNC'] * df['RAIL_FLAG']

df['dockless_flag'] = df['dockCt'].apply(lambda x : min(x,1))
df['scooter_flag']  = df['scooterCt'].apply(lambda x : min(x,1))

df['BIKE_SHARE'] = df['PBS_Flag'] + df['dockless_flag']
df['BIKE_SHARE'] = df['BIKE_SHARE'].apply(lambda x : min(x,1))


df['PBS_Flag_BUS']   = df['PBS_Flag']   * df['BUS_FLAG']
df['dockCt_BUS']     = df['dockCt']     * df['BUS_FLAG']
df['docklessCt_BUS'] = df['docklessCt'] * df['BUS_FLAG']
df['scooterCt_BUS']  = df['scooterCt']  * df['BUS_FLAG']
df['dockless_flag_BUS'] = df['dockless_flag'] * df['BUS_FLAG']
df['scooter_flag_BUS']  = df['scooter_flag']  * df['BUS_FLAG']
df['BIKE_SHARE_BUS']  = df['BIKE_SHARE']  * df['BUS_FLAG']


df['PBS_Flag_RAIL']   = df['PBS_Flag']   * df['RAIL_FLAG']
df['dockCt_RAIL']     = df['dockCt']     * df['RAIL_FLAG']
df['docklessCt_RAIL'] = df['docklessCt'] * df['RAIL_FLAG']
df['scooterCt_RAIL']  = df['scooterCt']  * df['RAIL_FLAG']
df['dockless_flag_RAIL'] = df['dockless_flag'] * df['RAIL_FLAG']
df['scooter_flag_RAIL']  = df['scooter_flag']  * df['RAIL_FLAG']
df['BIKE_SHARE_RAIL']  = df['BIKE_SHARE']  * df['RAIL_FLAG']

In [24]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('#VALUE!', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('#VALUE!', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('#VALUE!', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('#VALUE!', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('#VALUE!', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('#VALUE!', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('#VALUE!', np.nan)


df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('N', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('N', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('N', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('N', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('N', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('N', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('N', np.nan)



In [25]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].astype(float)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].astype(float)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].astype(float)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].astype(float)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].astype(float)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].astype(float)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].astype(float)

In [26]:
# immigrant population percentage
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [27]:
# bike share plus scooters for rail
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# merge bike and scooter for bus
df['BIKE_SCOOTER_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] + df['scooter_flag_BUS']

# merge bike and scooter for bus
df['BIKE_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 

# scooter or bike
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_OR_SCOOTER_BUS'].apply(lambda x : min(x, 1))

df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 
df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_OR_SCOOTER_RAIL'].apply(lambda x : min(x, 1))

# calculate average vehicles
df['AVG_VEHS'] = (df['HH_1Veh'] + 2*df['HH_2Veh'] + 3*df['HH_3Veh'] + 4*df['HH_4+Veh']) / (df['HH_0Veh'] + df['HH_1Veh'] + df['HH_2Veh'] + df['HH_3Veh'] + df['HH_4+Veh'])
df['AVG_VEHS_log'] = np.log(1+df['AVG_VEHS'])

In [28]:
# walk bike other commuting
df['JTW_WALK_BIKE_OTHER_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT'] + df['JTW_OTHER_PCT']

# walk bike commuting
df['JTW_WALK_BIKE_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT']

In [29]:
df['VRM_ADJ_BUS'] = df['VRM_ADJ'] * df['BUS_FLAG']
df['VRM_ADJ_RAIL'] = df['VRM_ADJ'] * df['RAIL_FLAG']


#maintenance effect on competing mode
df['MDBF_Mechanical_RAIL']  = df['MDBF_Mechanical'] * df['RAIL_FLAG']
df['MDBF_Mechanical_BUS'] = df['MDBF_Mechanical']  * df['BUS_FLAG']

df['MDBF_Total_RAIL']  = df['MDBF_Total'] * df['RAIL_FLAG']
df['MDBF_Total_BUS'] = df['MDBF_Total']  * df['BUS_FLAG']

In [30]:
# segment TNCs by cluster
df['HI_OPEX']  = np.where(df['CLUSTER_APTA4']==1, 1, 0)
df['MID_OPEX'] = np.where(df['CLUSTER_APTA4']==2, 1, 0)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['NEW_YORK'] = np.where(df['CLUSTER_APTA4']==10, 1, 0)

df['YEARS_SINCE_TNC_BUS_HI']  = df['YEARS_SINCE_TNC_BUS'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS_MID'] = df['YEARS_SINCE_TNC_BUS'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS_LOW'] = df['YEARS_SINCE_TNC_BUS'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_BUS_NY']  = df['YEARS_SINCE_TNC_BUS'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_RAIL_HI']  = df['YEARS_SINCE_TNC_RAIL'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL_MID'] = df['YEARS_SINCE_TNC_RAIL'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL_LOW'] = df['YEARS_SINCE_TNC_RAIL'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_RAIL_NY']  = df['YEARS_SINCE_TNC_RAIL'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_BUS_HINY']  = df['YEARS_SINCE_TNC_BUS_HI'] + df['YEARS_SINCE_TNC_BUS_NY']
df['YEARS_SINCE_TNC_RAIL_HINY']  = df['YEARS_SINCE_TNC_RAIL_HI'] + df['YEARS_SINCE_TNC_RAIL_NY']

df['YEARS_SINCE_TNC_BUS_MIDLOW']  = df['YEARS_SINCE_TNC_BUS_MID'] + df['YEARS_SINCE_TNC_BUS_LOW']
df['YEARS_SINCE_TNC_RAIL_MIDLOW']  = df['YEARS_SINCE_TNC_RAIL_MID'] + df['YEARS_SINCE_TNC_RAIL_LOW']

In [31]:
# bikes and scooters by segment

df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']
df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']

df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']
df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']


df['BIKE_SHARE_HINY']  = df['BIKE_SHARE_HI'] + df['BIKE_SHARE_NY']
df['BIKE_SHARE_MIDLOW']  = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_HINY']  = df['scooter_flag_HI'] + df['scooter_flag_NY']
df['scooter_flag_MIDLOW']  = df['scooter_flag_MID'] + df['scooter_flag_LOW']

In [32]:
tnc_rev=pd.read_csv('tnc_revenue.csv',encoding='utf-8')
tnc_rev.head()

,Unnamed: 0,CBSA,MNAME,Mode,Year,TNC_FLAG,YEARS_SINCE_TNC,UBER_NET_REVENUE,UBER_REV_SCALED
0,0,10420,"Akron, OH Metro Area",Bus,2014,1,0,445,1.000000
1,1,10420,"Akron, OH Metro Area",Bus,2015,1,1,1690,3.797753
2,2,10420,"Akron, OH Metro Area",Bus,2016,1,2,4964,11.155056
3,3,10420,"Akron, OH Metro Area",Bus,2017,1,3,7932,17.824719
4,4,10420,"Akron, OH Metro Area",Bus,2018,1,4,11270,25.325843


In [33]:
tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)

In [34]:
list(tnc_rev)

['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'UBER_NET_REVENUE',
 'UBER_REV_SCALED']

In [35]:
df=pd.merge(df,tnc_rev,on=['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'TNC_FLAG',
 'YEARS_SINCE_TNC'],how='left')
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_EMP_PCT_LEHD,TSD_EMP_PCT_CBSA,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,LOW_OPEX,RAIL_FLAG,LOW_OPEX_RAIL,YearDiff,BUS_FLAG,POP_EMP,HH_EMP,Tot_NonUSA_POP_pct,YEARS_SINCE_2002,YEARS_2002_2010,YEARS_AFTER_2010,YEARS_2002_2014,YEARS_AFTER_2014,YEARS_SINCE_2002_BUS,YEARS_SINCE_2002_RAIL,YEARS_2002_2010_BUS,YEARS_AFTER_2010_BUS,YEARS_2002_2010_RAIL,YEARS_AFTER_2010_RAIL,YEARS_2002_2014_BUS,YEARS_AFTER_2014_BUS,YEARS_2002_2014_RAIL,YEARS_AFTER_2014_RAIL,YEARS_AFTER_2012,TNC_YEARS_AFTER_2012,TNC_YEARS_AFTER_2012_BUS,TNC_YEARS_AFTER_2012_RAIL,YEARS_SINCE_TNC_2012,YEARS_SINCE_TNC_2012_BUS,YEARS_SINCE_TNC_2012_RAIL,YEARS_SINCE_RAIL_START,YEARS_SINCE_RAIL_START_1,YEARS_SINCE_RAIL_START_2,YEARS_SINCE_RAIL_START_3,TNC_FLAG_BUS,TNC_FLAG_RAIL,YEARS_SINCE_TNC_BUS,YEARS_SINCE_TNC_RAIL,dockless_flag,scooter_flag,BIKE_SHARE,PBS_Flag_BUS,dockCt_BUS,docklessCt_BUS,scooterCt_BUS,dockless_flag_BUS,scooter_flag_BUS,BIKE_SHARE_BUS,PBS_Flag_RAIL,dockCt_RAIL,docklessCt_RAIL,scooterCt_RAIL,dockless_flag_RAIL,scooter_flag_RAIL,BIKE_SHARE_RAIL,BIKE_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_BUS,BIKE_SCOOTER_BUS,BIKE_OR_SCOOTER_BUS,BIKE_OR_SCOOTER_RAIL,AVG_VEHS,AVG_VEHS_log,JTW_WALK_BIKE_OTHER_PCT,JTW_WALK_BIKE_PCT,VRM_ADJ_BUS,VRM_ADJ_RAIL,MDBF_Mechanical_RAIL,MDBF_Mechanical_BUS,MDBF_Total_RAIL,MDBF_Total_BUS,HI_OPEX,MID_OPEX,NEW_YORK,YEARS_SINCE_TNC_BUS_HI,YEARS_SINCE_TNC_BUS_MID,YEARS_SINCE_TNC_BUS_LOW,YEARS_SINCE_TNC_BUS_NY,YEARS_SINCE_TNC_RAIL_HI,YEARS_SINCE_TNC_RAIL_MID,YEARS_SINCE_TNC_RAIL_LOW,YEARS_SINCE_TNC_R

In [36]:
df['UBER_NET_REVENUE']=df['UBER_NET_REVENUE'].fillna(0)
df['UBER_REV_SCALED']=df['UBER_REV_SCALED'].fillna(0)
df.head(20)

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_EMP_PCT_LEHD,TSD_EMP_PCT_CBSA,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,LOW_OPEX,RAIL_FLAG,LOW_OPEX_RAIL,YearDiff,BUS_FLAG,POP_EMP,HH_EMP,Tot_NonUSA_POP_pct,YEARS_SINCE_2002,YEARS_2002_2010,YEARS_AFTER_2010,YEARS_2002_2014,YEARS_AFTER_2014,YEARS_SINCE_2002_BUS,YEARS_SINCE_2002_RAIL,YEARS_2002_2010_BUS,YEARS_AFTER_2010_BUS,YEARS_2002_2010_RAIL,YEARS_AFTER_2010_RAIL,YEARS_2002_2014_BUS,YEARS_AFTER_2014_BUS,YEARS_2002_2014_RAIL,YEARS_AFTER_2014_RAIL,YEARS_AFTER_2012,TNC_YEARS_AFTER_2012,TNC_YEARS_AFTER_2012_BUS,TNC_YEARS_AFTER_2012_RAIL,YEARS_SINCE_TNC_2012,YEARS_SINCE_TNC_2012_BUS,YEARS_SINCE_TNC_2012_RAIL,YEARS_SINCE_RAIL_START,YEARS_SINCE_RAIL_START_1,YEARS_SINCE_RAIL_START_2,YEARS_SINCE_RAIL_START_3,TNC_FLAG_BUS,TNC_FLAG_RAIL,YEARS_SINCE_TNC_BUS,YEARS_SINCE_TNC_RAIL,dockless_flag,scooter_flag,BIKE_SHARE,PBS_Flag_BUS,dockCt_BUS,docklessCt_BUS,scooterCt_BUS,dockless_flag_BUS,scooter_flag_BUS,BIKE_SHARE_BUS,PBS_Flag_RAIL,dockCt_RAIL,docklessCt_RAIL,scooterCt_RAIL,dockless_flag_RAIL,scooter_flag_RAIL,BIKE_SHARE_RAIL,BIKE_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_BUS,BIKE_SCOOTER_BUS,BIKE_OR_SCOOTER_BUS,BIKE_OR_SCOOTER_RAIL,AVG_VEHS,AVG_VEHS_log,JTW_WALK_BIKE_OTHER_PCT,JTW_WALK_BIKE_PCT,VRM_ADJ_BUS,VRM_ADJ_RAIL,MDBF_Mechanical_RAIL,MDBF_Mechanical_BUS,MDBF_Total_RAIL,MDBF_Total_BUS,HI_OPEX,MID_OPEX,NEW_YORK,YEARS_SINCE_TNC_BUS_HI,YEARS_SINCE_TNC_BUS_MID,YEARS_SINCE_TNC_BUS_LOW,YEARS_SINCE_TNC_BUS_NY,YEARS_SINCE_TNC_RAIL_HI,YEARS_SINCE_TNC_RAIL_MID,YEARS_SINCE_TNC_RAIL_LOW,YEARS_SINCE_TNC_R

In [37]:
df['UBER_REV_SCALED_BUS'] = df['UBER_REV_SCALED'] * df['BUS_FLAG']
df['UBER_REV_SCALED_RAIL'] = df['UBER_REV_SCALED'] * df['RAIL_FLAG']

df['UBER_REV_SCALED_BUS_HI']  = df['UBER_REV_SCALED_BUS'] * df['HI_OPEX']
df['UBER_REV_SCALED_BUS_MID'] = df['UBER_REV_SCALED_BUS'] * df['MID_OPEX']
df['UBER_REV_SCALED_BUS_LOW'] = df['UBER_REV_SCALED_BUS'] * df['LOW_OPEX']
df['UBER_REV_SCALED_BUS_NY']  = df['UBER_REV_SCALED_BUS'] * df['NEW_YORK']

df['UBER_REV_SCALED_RAIL_HI']  = df['UBER_REV_SCALED_RAIL'] * df['HI_OPEX']
df['UBER_REV_SCALED_RAIL_MID'] = df['UBER_REV_SCALED_RAIL'] * df['MID_OPEX']
df['UBER_REV_SCALED_RAIL_LOW'] = df['UBER_REV_SCALED_RAIL'] * df['LOW_OPEX']
df['UBER_REV_SCALED_RAIL_NY']  = df['UBER_REV_SCALED_RAIL'] * df['NEW_YORK']

df['UBER_REV_SCALED_BUS_HINY']  = df['UBER_REV_SCALED_BUS_HI'] + df['UBER_REV_SCALED_BUS_NY']
df['UBER_REV_SCALED_RAIL_HINY']  = df['UBER_REV_SCALED_RAIL_HI'] + df['UBER_REV_SCALED_RAIL_NY']

df['UBER_REV_SCALED_BUS_MIDLOW']  = df['UBER_REV_SCALED_BUS_MID'] + df['UBER_REV_SCALED_BUS_LOW']
df['UBER_REV_SCALED_RAIL_MIDLOW']  = df['UBER_REV_SCALED_RAIL_MID'] + df['UBER_REV_SCALED_RAIL_LOW']

In [38]:
df['UBER_NET_REVENUE_BUS'] = df['UBER_NET_REVENUE'] * df['BUS_FLAG']
df['UBER_NET_REVENUE_RAIL'] = df['UBER_NET_REVENUE'] * df['RAIL_FLAG']

df['UBER_NET_REVENUE_BUS_HI']  = df['UBER_NET_REVENUE_BUS'] * df['HI_OPEX']
df['UBER_NET_REVENUE_BUS_MID'] = df['UBER_NET_REVENUE_BUS'] * df['MID_OPEX']
df['UBER_NET_REVENUE_BUS_LOW'] = df['UBER_NET_REVENUE_BUS'] * df['LOW_OPEX']
df['UBER_NET_REVENUE_BUS_NY']  = df['UBER_NET_REVENUE_BUS'] * df['NEW_YORK']

df['UBER_NET_REVENUE_RAIL_HI']  = df['UBER_NET_REVENUE_RAIL'] * df['HI_OPEX']
df['UBER_NET_REVENUE_RAIL_MID'] = df['UBER_NET_REVENUE_RAIL'] * df['MID_OPEX']
df['UBER_NET_REVENUE_RAIL_LOW'] = df['UBER_NET_REVENUE_RAIL'] * df['LOW_OPEX']
df['UBER_NET_REVENUE_RAIL_NY']  = df['UBER_NET_REVENUE_RAIL'] * df['NEW_YORK']

df['UBER_NET_REVENUE_BUS_HINY']  = df['UBER_NET_REVENUE_BUS_HI'] + df['UBER_NET_REVENUE_BUS_NY']
df['UBER_NET_REVENUE_RAIL_HINY']  = df['UBER_NET_REVENUE_RAIL_HI'] + df['UBER_NET_REVENUE_RAIL_NY']

df['UBER_NET_REVENUE_BUS_MIDLOW']  = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_LOW']
df['UBER_NET_REVENUE_RAIL_MIDLOW']  = df['UBER_NET_REVENUE_RAIL_MID'] + df['UBER_NET_REVENUE_RAIL_LOW']

In [39]:
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOT_EMP_TSD_TRACTS,TOT_EMP_ALL_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_EMP_PCT_LEHD,TSD_EMP_PCT_CBSA,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,LOW_OPEX,RAIL_FLAG,LOW_OPEX_RAIL,YearDiff,BUS_FLAG,POP_EMP,HH_EMP,Tot_NonUSA_POP_pct,YEARS_SINCE_2002,YEARS_2002_2010,YEARS_AFTER_2010,YEARS_2002_2014,YEARS_AFTER_2014,YEARS_SINCE_2002_BUS,YEARS_SINCE_2002_RAIL,YEARS_2002_2010_BUS,YEARS_AFTER_2010_BUS,YEARS_2002_2010_RAIL,YEARS_AFTER_2010_RAIL,YEARS_2002_2014_BUS,YEARS_AFTER_2014_BUS,YEARS_2002_2014_RAIL,YEARS_AFTER_2014_RAIL,YEARS_AFTER_2012,TNC_YEARS_AFTER_2012,TNC_YEARS_AFTER_2012_BUS,TNC_YEARS_AFTER_2012_RAIL,YEARS_SINCE_TNC_2012,YEARS_SINCE_TNC_2012_BUS,YEARS_SINCE_TNC_2012_RAIL,YEARS_SINCE_RAIL_START,YEARS_SINCE_RAIL_START_1,YEARS_SINCE_RAIL_START_2,YEARS_SINCE_RAIL_START_3,TNC_FLAG_BUS,TNC_FLAG_RAIL,YEARS_SINCE_TNC_BUS,YEARS_SINCE_TNC_RAIL,dockless_flag,scooter_flag,BIKE_SHARE,PBS_Flag_BUS,dockCt_BUS,docklessCt_BUS,scooterCt_BUS,dockless_flag_BUS,scooter_flag_BUS,BIKE_SHARE_BUS,PBS_Flag_RAIL,dockCt_RAIL,docklessCt_RAIL,scooterCt_RAIL,dockless_flag_RAIL,scooter_flag_RAIL,BIKE_SHARE_RAIL,BIKE_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_RAIL,BIKE_SCOOTER_SCOOTER_BUS,BIKE_SCOOTER_BUS,BIKE_OR_SCOOTER_BUS,BIKE_OR_SCOOTER_RAIL,AVG_VEHS,AVG_VEHS_log,JTW_WALK_BIKE_OTHER_PCT,JTW_WALK_BIKE_PCT,VRM_ADJ_BUS,VRM_ADJ_RAIL,MDBF_Mechanical_RAIL,MDBF_Mechanical_BUS,MDBF_Total_RAIL,MDBF_Total_BUS,HI_OPEX,MID_OPEX,NEW_YORK,YEARS_SINCE_TNC_BUS_HI,YEARS_SINCE_TNC_BUS_MID,YEARS_SINCE_TNC_BUS_LOW,YEARS_SINCE_TNC_BUS_NY,YEARS_SINCE_TNC_RAIL_HI,YEARS_SINCE_TNC_RAIL_MID,YEARS_SINCE_TNC_RAIL_LOW,YEARS_SINCE_TNC_R

In [40]:
# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

In [41]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [42]:
# create a log of all fields
for col in df.columns:
    df[col+'_log'] = np.log(df[col]+1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [43]:
# Test TNC ramp up

df['YEARS_SINCE_TNC_BUS_2'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_BUS_3'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_BUS_4'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_BUS_5'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 5))

df['YEARS_SINCE_TNC_RAIL_2'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_RAIL_3'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_RAIL_4'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_RAIL_5'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 5))

In [44]:
# count the first year of TNC presence
df['YEARS_SINCE_TNC_BUS2'] = df['YEARS_SINCE_TNC_BUS'] + df['TNC_FLAG_BUS']
df['YEARS_SINCE_TNC_RAIL2'] = df['YEARS_SINCE_TNC_RAIL'] + df['TNC_FLAG_RAIL']


In [45]:
## start in first year TNCs show up

df['YEARS_SINCE_TNC_BUS2_NY']  = df['YEARS_SINCE_TNC_BUS2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_BUS2_HI']  = df['YEARS_SINCE_TNC_BUS2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS2_MID'] = df['YEARS_SINCE_TNC_BUS2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS2_LOW'] = df['YEARS_SINCE_TNC_BUS2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_BUS2_HINY']   = df['YEARS_SINCE_TNC_BUS2_NY'] + df['YEARS_SINCE_TNC_BUS2_HI']
df['YEARS_SINCE_TNC_BUS2_MIDLOW'] = df['YEARS_SINCE_TNC_BUS2_MID'] + df['YEARS_SINCE_TNC_BUS2_LOW']

df['YEARS_SINCE_TNC_RAIL2_NY']  = df['YEARS_SINCE_TNC_RAIL2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_RAIL2_HI']  = df['YEARS_SINCE_TNC_RAIL2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL2_MID'] = df['YEARS_SINCE_TNC_RAIL2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL2_LOW'] = df['YEARS_SINCE_TNC_RAIL2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_RAIL2_HINY']   = df['YEARS_SINCE_TNC_RAIL2_NY'] + df['YEARS_SINCE_TNC_RAIL2_HI']
df['YEARS_SINCE_TNC_RAIL2_MIDLOW'] = df['YEARS_SINCE_TNC_RAIL2_MID'] + df['YEARS_SINCE_TNC_RAIL2_LOW']

In [46]:
## BIKE SHARE SEGMENTATION

df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']
df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']

df['BIKE_SHARE_HINY']   = df['BIKE_SHARE_NY'] + df['BIKE_SHARE_HI']
df['BIKE_SHARE_MIDLOW'] = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']
df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']

df['scooter_flag_HINY']   = df['scooter_flag_NY'] + df['scooter_flag_HI']
df['scooter_flag_MIDLOW'] = df['scooter_flag_MID'] + df['scooter_flag_LOW']



## Model Estimation

In [47]:
# base model 1 from 9-5
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6406
Estimator:                   PanelOLS   R-squared (Between):              0.9726
No. Observations:                4097   R-squared (Within):               0.6406
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9731
Time:                        15:36:11   Log-likelihood                    1353.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      566.79
Entities:                         269   P-value                           0.0000
Avg Obs:                       15.230   Distribution:                 F(12,3816)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             566.79
                            

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


### Years since TNC

In [48]:
# switch to LEHD population percent

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6452
Estimator:                   PanelOLS   R-squared (Between):              0.9314
No. Observations:                3985   R-squared (Within):               0.6452
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9320
Time:                        15:36:31   Log-likelihood                    1340.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      562.53
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(12,3712)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             562.53
                            

In [49]:
#TNC years after 2012 -- A little better

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6455
Estimator:                   PanelOLS   R-squared (Between):              0.9349
No. Observations:                3985   R-squared (Within):               0.6455
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9354
Time:                        15:36:38   Log-likelihood                    1342.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      563.23
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(12,3712)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             563.23
                            

In [50]:
# exponentiated years since TNC

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + np.exp(YEARS_SINCE_TNC_BUS)\
                    + np.exp(YEARS_SINCE_TNC_RAIL) \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6432
Estimator:                   PanelOLS   R-squared (Between):              0.9016
No. Observations:                3985   R-squared (Within):               0.6432
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9024
Time:                        15:36:45   Log-likelihood                    1329.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      557.55
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(12,3712)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             557.55
                            

### Uber Scaled Revenue

In [51]:
# switch to Uber scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_REV_SCALED_BUS \
                    + UBER_REV_SCALED_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.8994
No. Observations:                3985   R-squared (Within):               0.6437
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9003
Time:                        15:36:52   Log-likelihood                    1332.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      558.80
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(12,3712)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             558.80
                            

In [52]:
# switch to Uber scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_REV_SCALED_BUS_HINY \
                    + UBER_REV_SCALED_BUS_MIDLOW \
                    + UBER_REV_SCALED_RAIL_HINY \
                    + UBER_REV_SCALED_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6455
Estimator:                   PanelOLS   R-squared (Between):              0.9166
No. Observations:                3985   R-squared (Within):               0.6455
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9173
Time:                        15:37:01   Log-likelihood                    1342.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      482.54
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(14,3710)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             482.54
                            

### Uber un-scaled revenue

In [53]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS \
                    + UBER_NET_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6462
Estimator:                   PanelOLS   R-squared (Between):              0.9487
No. Observations:                3985   R-squared (Within):               0.6462
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9491
Time:                        15:37:20   Log-likelihood                    1346.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      564.99
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(12,3712)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             564.99
                            

In [54]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MIDLOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6462
Estimator:                   PanelOLS   R-squared (Between):              0.9487
No. Observations:                3985   R-squared (Within):               0.6462
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9491
Time:                        15:37:49   Log-likelihood                    1346.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      484.09
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(14,3710)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             484.09
                            

In [56]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6476
Estimator:                   PanelOLS   R-squared (Between):              0.9496
No. Observations:                3985   R-squared (Within):               0.6476
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9500
Time:                        15:38:18   Log-likelihood                    1354.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      454.41
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(15,3709)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             454.41
                            

### Separate Rail vs bus models

In [57]:
# separate bus model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6877
Estimator:                   PanelOLS   R-squared (Between):              0.9898
No. Observations:                3412   R-squared (Within):               0.6877
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9897
Time:                        15:40:01   Log-likelihood                    1481.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      538.48
Entities:                         220   P-value                           0.0000
Avg Obs:                       15.509   Distribution:                 F(13,3179)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.48
                            

In [59]:
# separate rail model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5843
Estimator:                   PanelOLS   R-squared (Between):              0.8460
No. Observations:                 573   R-squared (Within):               0.5843
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.8464
Time:                        15:40:45   Log-likelihood                    52.736
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      60.920
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(12,520)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             60.920
                            

### Separate models by cluster

In [88]:
# combined model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6475
Estimator:                   PanelOLS   R-squared (Between):              0.9490
No. Observations:                3993   R-squared (Within):               0.6475
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9494
Time:                        16:30:37   Log-likelihood                    1359.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.20
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.20
                            

In [89]:
# high op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[(df['HI_OPEX']==1) | (df['NEW_YORK']==1)])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.8637
Estimator:                   PanelOLS   R-squared (Between):              0.3635
No. Observations:                 658   R-squared (Within):               0.8637
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.3661
Time:                        16:31:07   Log-likelihood                    341.57
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      320.14
Entities:                          40   P-value                           0.0000
Avg Obs:                       16.450   Distribution:                  F(12,606)
Min Obs:                      10.0000                                           
Max Obs:                       17.000   F-statistic (robust):             320.14
                            

In [90]:
# mid op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS \
                    + UBER_NET_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[df['MID_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5892
Estimator:                   PanelOLS   R-squared (Between):              0.8850
No. Observations:                1305   R-squared (Within):               0.5892
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.8840
Time:                        16:31:28   Log-likelihood                    512.59
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      144.40
Entities:                          85   P-value                           0.0000
Avg Obs:                       15.353   Distribution:                 F(12,1208)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             144.40
                            

In [91]:
# low op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6386
Estimator:                   PanelOLS   R-squared (Between):              0.9859
No. Observations:                2030   R-squared (Within):               0.6386
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9858
Time:                        16:31:49   Log-likelihood                    791.66
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      302.47
Entities:                         136   P-value                           0.0000
Avg Obs:                       14.926   Distribution:                 F(11,1883)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             302.47
                            

In [92]:
## excluding low op-ex, since they are different

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==0])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6732
Estimator:                   PanelOLS   R-squared (Between):              0.9387
No. Observations:                1963   R-squared (Within):               0.6732
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9393
Time:                        16:32:08   Log-likelihood                    631.02
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      268.41
Entities:                         125   P-value                           0.0000
Avg Obs:                       15.704   Distribution:                 F(14,1824)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             268.41
                            

In [93]:
# segment bus vs rail and low vs everything else
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5891
Estimator:                   PanelOLS   R-squared (Between):              0.8464
No. Observations:                 573   R-squared (Within):               0.5891
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.8468
Time:                        16:32:27   Log-likelihood                    56.023
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      67.901
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             67.901
                            

In [94]:
# segment bus vs rail and low vs everything else
# bus -- mid and above

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==0])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6718
Estimator:                   PanelOLS   R-squared (Between):              0.9358
No. Observations:                1963   R-squared (Within):               0.6718
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9365
Time:                        16:32:46   Log-likelihood                    626.63
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      339.90
Entities:                         125   P-value                           0.0000
Avg Obs:                       15.704   Distribution:                 F(11,1827)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             339.90
                            

In [96]:
# segment bus vs rail and low vs everything else
# bus -- low op-ex

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6386
Estimator:                   PanelOLS   R-squared (Between):              0.9859
No. Observations:                2030   R-squared (Within):               0.6386
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9858
Time:                        16:33:43   Log-likelihood                    791.66
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      302.47
Entities:                         136   P-value                           0.0000
Avg Obs:                       14.926   Distribution:                 F(11,1883)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             302.47
                            

### Currently prefer the combined model

In [68]:
# combined model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6476
Estimator:                   PanelOLS   R-squared (Between):              0.9496
No. Observations:                3985   R-squared (Within):               0.6476
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9500
Time:                        15:53:09   Log-likelihood                    1354.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      454.41
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(15,3709)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             454.41
                            

In [70]:
# add maintenance
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6477
Estimator:                   PanelOLS   R-squared (Between):              0.9487
No. Observations:                3985   R-squared (Within):               0.6477
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9491
Time:                        15:54:18   Log-likelihood                    1354.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      426.03
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.268   Distribution:                 F(16,3708)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             426.03
                            

In [71]:
# drop work at home
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6475
Estimator:                   PanelOLS   R-squared (Between):              0.9490
No. Observations:                3993   R-squared (Within):               0.6475
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9494
Time:                        15:56:00   Log-likelihood                    1359.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.20
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.20
                            

In [72]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6477
Estimator:                   PanelOLS   R-squared (Between):              0.9499
No. Observations:                3993   R-squared (Within):               0.6477
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9502
Time:                        15:56:34   Log-likelihood                    1360.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.65
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.65
                            

In [74]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6476
Estimator:                   PanelOLS   R-squared (Between):              0.9485
No. Observations:                3993   R-squared (Within):               0.6476
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9489
Time:                        15:57:17   Log-likelihood                    1360.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.31
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.31
                            

In [75]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6476
Estimator:                   PanelOLS   R-squared (Between):              0.9494
No. Observations:                3993   R-squared (Within):               0.6476
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9498
Time:                        15:57:33   Log-likelihood                    1359.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.31
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.31
                            

In [78]:
# combined

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_HINY \
                    + UBER_NET_REVENUE_BUS_MID \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL_HINY \
                    + UBER_NET_REVENUE_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6476
Estimator:                   PanelOLS   R-squared (Between):              0.9485
No. Observations:                3993   R-squared (Within):               0.6476
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9489
Time:                        15:59:29   Log-likelihood                    1360.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      455.31
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(15,3717)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             455.31
                            

In [98]:
# slightly less segmentation
df['UBER_NET_REVENUE_BUS_MIDHINY'] = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_HINY']

In [101]:
# current winner

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_MIDHINY \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + UBER_NET_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6475
Estimator:                   PanelOLS   R-squared (Between):              0.9490
No. Observations:                3993   R-squared (Within):               0.6475
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9494
Time:                        16:40:07   Log-likelihood                    1359.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      525.47
Entities:                         261   P-value                           0.0000
Avg Obs:                       15.299   Distribution:                 F(13,3719)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             525.47
                            

### Bus vs rail with just TNCs segmented by op-ex

In [112]:
# bus

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_MIDHINY \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6875
Estimator:                   PanelOLS   R-squared (Between):              0.9898
No. Observations:                3420   R-squared (Within):               0.6875
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9897
Time:                        16:54:35   Log-likelihood                    1487.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      637.69
Entities:                         220   P-value                           0.0000
Avg Obs:                       15.545   Distribution:                 F(11,3189)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             637.69
                            

In [107]:
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5836
Estimator:                   PanelOLS   R-squared (Between):              0.8829
No. Observations:                 573   R-squared (Within):               0.5836
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.8827
Time:                        16:51:27   Log-likelihood                    52.236
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      66.388
Entities:                          41   P-value                           0.0000
Avg Obs:                       13.976   Distribution:                  F(11,521)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             66.388
                            

### Current best models below, split bus vs rail

In [115]:
# bus - simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + UBER_NET_REVENUE_BUS_MIDHINY \
                    + UBER_NET_REVENUE_BUS_LOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6875
Estimator:                   PanelOLS   R-squared (Between):              0.9898
No. Observations:                3420   R-squared (Within):               0.6875
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.9897
Time:                        16:57:17   Log-likelihood                    1487.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      637.69
Entities:                         220   P-value                           0.0000
Avg Obs:                       15.545   Distribution:                 F(11,3189)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             637.69
                            

In [120]:
# rail -- simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + UBER_NET_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5799
Estimator:                   PanelOLS   R-squared (Between):              0.8447
No. Observations:                 574   R-squared (Within):               0.5799
Date:                Mon, Apr 20 2020   R-squared (Overall):              0.8457
Time:                        17:02:05   Log-likelihood                    50.217
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      80.370
Entities:                         258   P-value                           0.0000
Avg Obs:                       2.2248   Distribution:                   F(9,524)
Min Obs:                       0.0000                                           
Max Obs:                       17.000   F-statistic (robust):             80.370
                            